In [2]:
from datetime import datetime
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly
import plotly.graph_objects as go
import time
from matplotlib.cm import get_cmap
from scipy.spatial.distance import cdist
import os
import concurrent.futures
from matplotlib.cm import get_cmap
from tqdm import tqdm
from joblib import Parallel, delayed
import pathlib

print("hello")
datapath = "data/"

lat_river_mouth, lon_river_mouth = 41.139024, -8.680089
figpath = "fig/"
filepath = "raw/Nov/"
files = os.listdir(filepath)
files.sort()


hello


In [4]:
path_wind = "wind/raw/wind_data.txt"
path_tide = "tide/raw/Month_11.csv"

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams.update({'font.size': 12})
plt.rcParams.update({'font.style': 'oblique'})


wind = np.loadtxt(path_wind, delimiter=",")
tide = pd.read_csv(path_tide).to_numpy()
# tide = np.loadtxt(path_tide, delimiter=", ")

def angle2anle(angle):
    return 270 - angle

def s2uv(s, angle):
    angle = angle2anle(angle)
    u = s * np.cos(deg2rad(angle))
    v = s * np.sin(deg2rad(angle))
    return u, v

def rad2deg(rad):
    return rad / np.pi * 180

def deg2rad(deg):
    return deg / 180 * np.pi

def checkfolder(folder):
    path = pathlib.Path(folder)
    path.mkdir(parents=True, exist_ok=True)
#     print(folder + "is created")


In [7]:
def plot_files(filename):
    print(filename)
    date_string = filename[-14:-10] + filename[-6:-4]
    data = h5py.File(filepath+filename, 'r')
    t1 = time.time()
    df = data.get('data')
    lon = np.array(df.get("X")).squeeze()
    lat = np.array(df.get("Y")).squeeze()
    depth = np.array(df.get("Z"))
    Time = np.array(df.get('Time'))
    timestamp = (Time - 719529) * 24 * 3600  # 719529 is how many days have passed from Jan1 0,
    # to Jan1 1970. Since 1970Jan1, is used as the starting index for datetime
    sal_data = np.array(df["Val"])
    t2 = time.time()
    print("Loading time takes: ", t2 - t1)

    timestamp_data = timestamp[0]
    counter = 0
    folder = figpath + "P_" + date_string + "/"
    checkfolder(folder)

    for i in tqdm(range(sal_data.shape[3])):
        plt.figure(figsize=(10, 10))
        im = plt.scatter(lon[0, :, :], lat[0, :, :], c=sal_data[0, :, :, i], vmin=15, vmax=36, cmap=get_cmap("BrBG", 10))
        dist_wind = np.abs(timestamp_data[i] - wind[:, 0])
        ind_wind = np.where(dist_wind == np.nanmin(dist_wind))[0][0]

        dist_tide = np.abs(timestamp_data[i] - tide[:, 0])
        ind_tide = np.where(dist_tide == np.nanmin(dist_tide))[0][0]
        # ind_wind = 0
        u, v = s2uv(wind[ind_wind, 1], wind[ind_wind, 2])

        plt.quiver(lon_river_mouth, lat_river_mouth, u, v, scale = 25)
        if tide[ind_tide, 2] == 1:
            if timestamp_data[i] >= tide[ind_tide, 0]:
                plt.quiver(-8.65, 41.2, -1, 0)
                plt.text(-8.65, 41.205, 'Ebb tide')
            else:
                plt.quiver(-8.65, 41.2, 1, 0)
                plt.text(-8.65, 41.205, 'Flood tide')
        else:
            if timestamp_data[i] >= tide[ind_tide, 0]:
                plt.quiver(-8.65, 41.2, 1, 0)
                plt.text(-8.65, 41.205, 'Flood tide')
            else:
                plt.quiver(-8.65, 41.2, -1, 0)
                plt.text(-8.65, 41.205, 'Ebb tide')
            # plt.quiver(-8.65, 41.2, -1, 0)

        angles = np.arange(4) * 90 + 45
        directions = ['East', 'South', 'West', 'North']
        id = len(angles[angles < wind[ind_wind, 2]]) - 1
        plt.text(-8.7, 41.04, "Wind direction: " + directions[id])

        speeds = np.array([0, 2.5, 6])
        levels = ['Mild', 'Moderate', 'Heavy']
        id = len(speeds[speeds < wind[ind_wind, 1]]) - 1
        plt.text(-8.7, 41.035, "Wind level: " + levels[id])

        plt.xlabel("Lon [deg]")
        plt.ylabel("Lat [deg]")
        plt.title("Surface salinity on " + datetime.fromtimestamp(timestamp_data[i]).strftime("%Y%m%d - %H:%M"))
        plt.colorbar(im)
        
        plt.savefig(folder + "P_{:03d}.png".format(counter))
#         plt.savefig(figpath + "P_{:012d}.png".format(int(timestamp_data[i])))
        plt.close("all")
        counter = counter + 1


In [8]:
Parallel(n_jobs=3)(delayed(plot_files)(f) for f in files)

/usr/lib/python3/dist-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[None, None, None, None, None, None, None, None, None, None, None, None]